**Kod do lambdy**

In [ ]:
import json
import boto3

def lambda_handler(event, context):
    # Replace 'your-endpoint-name' with your SageMaker endpoint name
    endpoint_name = 'tensorflow-inference-2024-01-30-01-45-49-702'
    #endpoint_name = 'tensorflow-inference-2024-01-30-07-35-40-580'

    # The event contains the payload. In this case, it's a list of floats
    np_array = event['body']

    # Convert the NumPy array to the format your model expects
    payload = json.dumps(np_array)

    # Initialize boto3 client for SageMaker runtime
    runtime = boto3.client('runtime.sagemaker')

    # Send the payload to the SageMaker endpoint
    response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                       ContentType='application/json',
                                       Body=payload)

    # Parse the response
    result = json.loads(response['Body'].read().decode())

    return {
        'statusCode': 200,
        'body': json.dumps(result),
        'input': np_array
    }

**Interfejs do wysylania metody POST do Gateway API, ktore tzw. 'triggeruje' lambde**

In [2]:
import requests
import random
import json

def send_requests(endpoint_url, num_requests):
    responses = []

    for _ in range(num_requests):
        # Generate an array of 4 random floats between 0 and 1
        data = [random.random() for _ in range(4)]

        # Construct the JSON payload
        payload = {"body": data}
        #print(payload)

        # Send the POST request to the API endpoint
        response = requests.post(endpoint_url, json=payload)
        #print(response)

        # Parse the 'body' from the response
        if response.status_code == 200:
            #print(response.status_code)
            response_body = json.loads(response.json()['body'])

            # Extract the 'predictions' part of the response body
            prediction = response_body.get('predictions')
            responses.append(prediction)
        else:
            print(f"Request failed with status code {response.status_code}")

    return responses

# Example usage
endpoint = 'https://xw47mhnbmk.execute-api.eu-north-1.amazonaws.com/prodiris'
number_of_requests = 10  # Number of requests you want to send

collected_responses = send_requests(endpoint, number_of_requests)
print(collected_responses)


[[[0.328051537, 0.381023616, 0.290924847]], [[0.646202326, 0.248305678, 0.105491973]], [[0.246131316, 0.465190709, 0.28867802]], [[0.241114542, 0.442340642, 0.316544831]], [[0.117941096, 0.338157386, 0.543901503]], [[0.0976832, 0.388091654, 0.514225185]], [[0.168845445, 0.402555197, 0.428599417]], [[0.119567879, 0.37784794, 0.502584219]], [[0.122446798, 0.353894472, 0.523658752]], [[0.21576044, 0.354884803, 0.429354757]]]
